## Import Modules & Parameters

In [1]:
# import module
from mftool import Mftool
import pandas as pd, numpy as np
import urllib.request, warnings
from dateutil.relativedelta import relativedelta

warnings.filterwarnings("ignore")

In [12]:
target_url = 'https://raw.githubusercontent.com/NayakwadiS/mftool/master/Scheme_codes.txt'
rolling_return_yrs = 5

## User Defined Function

In [20]:
def get_rolling_returns_stats(scheme_name):
    
    # Get the Scheme Code
    for line in urllib.request.urlopen(target_url):
        if scheme_name.lower().replace(' ', '') in line.decode('utf-8').lower().replace(' ', ''):
            scheme_code = int(line.decode('utf-8').split(':')[0].replace("'", '').replace('\t', ''))
            break
    print("Scheme Name:", scheme_name)
    print("Scheme Code:", scheme_code)
    
    # pass the scheme code into methods
    obj = Mftool()
#     data = obj.get_scheme_quote(scheme_code)
    
    # Get Historical NAV and convert into pandas dataframe
    data = obj.get_scheme_historical_nav(scheme_code)
    nav_df = pd.DataFrame(data['data'])

    # Variable Type Change
    nav_df['nav'] = nav_df['nav'].astype(float)
    nav_df['date'] = pd.to_datetime(nav_df['date'], format='%d-%m-%Y')
    
    # Required Lag
    hist_cutoff_dt = max(nav_df['date']) - relativedelta(years=rolling_return_yrs)
    req_lag = nav_df.index[nav_df['date'] <= hist_cutoff_dt].tolist()[0]

    # Get Lagged Data
    nav_df['nav_lag'] = nav_df['nav'].shift(-req_lag)
    nav_df['date_lag'] = nav_df['date'].shift(-req_lag)

    # Calculate Return
    nav_df_sub = nav_df[nav_df['nav_lag'] > 0]
    nav_df_sub['return'] = (nav_df_sub['nav']/nav_df_sub['nav_lag']-1)/rolling_return_yrs
#     nav_df_sub['return'] = ((nav_df_sub['nav']/nav_df_sub['nav_lag'])^(1/rolling_return_yrs))-1
    
    # Compute Stats
    neg_reurn = nav_df_sub[nav_df_sub['return'] < 0].index.size/nav_df_sub.index.size
    ge_15_return = nav_df_sub[nav_df_sub['return'] >= 0.15].index.size/nav_df_sub.index.size
    le_10_return = nav_df_sub[nav_df_sub['return'] < 0.1].index.size/nav_df_sub.index.size

    # Print Stats
    print("    Max Return:", max(nav_df_sub['return'].dropna())*100, "%")
    print("    Avg Return:", np.mean(nav_df_sub['return'].dropna())*100, "%")
    print("    Min Return:", min(nav_df_sub['return'].dropna())*100, "%")
    print("    % times fund has given -ve return:", round(neg_reurn*100,2), "%")
    print("    % times fund has given >= 15% return:", round(ge_15_return*100,2), "%")
    print("    % times fund has given < 10% return:", round(le_10_return*100,2), "%")
    print("")

In [21]:
get_rolling_returns_stats('Axis Small Cap Fund - Direct Plan - Growth')
get_rolling_returns_stats('SBI Small Cap Fund - Direct Plan - Growth')
# get_rolling_returns_stats('SBI Blue Chip Fund - Direct Plan - Growth')

Scheme Name: Axis Small Cap Fund - Direct Plan - Growth
Scheme Code: 125354
    Max Return: 36.93386773547094 %
    Avg Return: 20.91883031792895 %
    Min Return: 5.232793522267203 %
    % times fund has given -ve return: 0.0 %
    % times fund has given >= 15% return: 80.03 %
    % times fund has given < 10% return: 8.33 %

Scheme Name: SBI Small Cap Fund - Direct Plan - Growth
Scheme Code: 125497
    Max Return: 63.30823482360542 %
    Avg Return: 30.00214064493362 %
    Min Return: 7.707651446048187 %
    % times fund has given -ve return: 0.0 %
    % times fund has given >= 15% return: 83.7 %
    % times fund has given < 10% return: 1.87 %

